<a href="https://colab.research.google.com/github/kroy900/Auto_Support_Resistance/blob/main/s_r_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mpl_finance
!pip install pandas_ta
!pip install yfinance
!pip install talib-binary

     |████████████████████████████████| 115 kB 5.1 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=f8559feeeeef9b6f7ea2bfd2e8c4a0af4e7949771c720b833d2f3a129af2782d
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta
     |████████████████████████████████| 6.4 MB 7.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but yo

In [21]:
import pandas as pd
import datetime as dt
import os
import numpy as np
import pandas_ta as ta
from pandas_datareader import data as pdr
import yfinance as yf
from talib import RSI
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
from numpy.ma.core import harden_mask
import plotly.graph_objects as go


plt.rcParams['figure.figsize'] = [12, 7]
plt.rc('font', size=14)


In [34]:
yf.pdr_override() 

filepath = r"sample_data/stock_list.xlsx"
stocklist = pd.read_excel(filepath)
stocklist=stocklist.head()

start =dt.datetime(2020,3,5)
now = dt.datetime.now()

for i in stocklist.index:
  stock=str(stocklist["Symbol"][i])
  df = pdr.get_data_yahoo(stock, start, now)
  print("Checking "+stock+".....")
  df['Date'] = pd.to_datetime(df.index)
  df['Date'] = df['Date'].apply(mpl_dates.date2num)
  df['Open'] = df['Open'].round(3)
  df['High'] = df['High'].round(3)
  df['Low'] = df['Low'].round(3)
  df['Close'] = df['Close'].round(3)
  df['Symbol'] = stock

# Reset Index and include date in fields. Date no longer Index
#df.reset_index(inplace=True)
  df = df.loc[:,['Symbol', 'Date', 'Open', 'High', 'Low', 'Close']]

# def EMA fast and slow parameters
  fast = 5
  slow = 12

# HMA fast and slow calculation
  df['src1'] = ta.hma(df['Open'], fast).shift(1).round(2)
  df['src2'] = ta.hma(df['Close'], slow).round(2)

  df['momm1'] = df.src1 - df.src1.shift(1)
  df['momm2'] = df.src2 - df.src2.shift(1)
  df['momm1'] = df['momm1'].round(2)
  df['momm2'] = df['momm2'].round(2)
#df.tail()

  def compare_values1(m,n):
    if m >= n:
      return m
    else:
      return 0.0  



  display(df)







[*********************100%***********************]  1 of 1 completed
Checking 20MICRONS.NS.....


,Symbol,Date,Open,High,Low,Close,src1,src2,momm1,momm2
Date,,,,,,,,,,
2020-03-05,20MICRONS.NS,737489.0,31.00,31.20,30.40,30.80,NaN,NaN,NaN,NaN
2020-03-06,20MICRONS.NS,737490.0,31.20,31.55,27.45,29.65,NaN,NaN,NaN,NaN
2020-03-09,20MICRONS.NS,737493.0,30.00,30.00,26.85,27.70,NaN,NaN,NaN,NaN
2020-03-11,20MICRONS.NS,737495.0,28.25,28.25,24.65,26.85,NaN,NaN,NaN,NaN
2020-03-12,20MICRONS.NS,737496.0,26.00,28.70,21.50,22.65,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-29,20MICRONS.NS,738274.0,83.30,85.50,80.10,81.00,86.16,83.66,-1.09,-1.69
2022-05-02,20MICRONS.NS,738277.0,80.15,84.20,79.45,83.05,83.77,82.40,-2.39,-1.26
2022-05-04,20MICRONS.NS,738279.0,84.70,84.70,77.55,79.90,80.12,81.06,-3.65,-1.34


[*********************100%***********************]  1 of 1 completed
Checking 21STCENMGM.NS.....


,Symbol,Date,Open,High,Low,Close,src1,src2,momm1,momm2
Date,,,,,,,,,,
2020-03-05,21STCENMGM.NS,737489.0,11.90,12.10,11.90,11.95,NaN,NaN,NaN,NaN
2020-03-06,21STCENMGM.NS,737490.0,12.00,12.05,11.75,12.05,NaN,NaN,NaN,NaN
2020-03-09,21STCENMGM.NS,737493.0,11.85,11.85,11.85,11.85,NaN,NaN,NaN,NaN
2020-03-11,21STCENMGM.NS,737495.0,11.85,11.85,11.85,11.85,NaN,NaN,NaN,NaN
2020-03-12,21STCENMGM.NS,737496.0,11.65,11.65,11.65,11.65,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-29,21STCENMGM.NS,738274.0,28.30,28.70,27.85,28.45,28.47,28.39,-1.31,-0.41
2022-05-02,21STCENMGM.NS,738277.0,28.00,29.00,28.00,28.95,27.69,28.24,-0.78,-0.15
2022-05-04,21STCENMGM.NS,738279.0,29.45,29.50,28.45,29.50,27.69,28.41,0.00,0.17


[*********************100%***********************]  1 of 1 completed
Checking 3IINFOLTD.NS.....


,Symbol,Date,Open,High,Low,Close,src1,src2,momm1,momm2
Date,,,,,,,,,,
2020-03-05,3IINFOLTD.NS,737489.0,18.50,19.50,18.50,19.00,NaN,NaN,NaN,NaN
2020-03-06,3IINFOLTD.NS,737490.0,18.50,18.50,17.00,17.50,NaN,NaN,NaN,NaN
2020-03-09,3IINFOLTD.NS,737493.0,17.50,18.00,16.50,17.00,NaN,NaN,NaN,NaN
2020-03-11,3IINFOLTD.NS,737495.0,17.50,18.00,16.50,17.50,NaN,NaN,NaN,NaN
2020-03-12,3IINFOLTD.NS,737496.0,17.00,17.00,16.00,16.50,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-29,3IINFOLTD.NS,738274.0,58.00,58.75,57.05,57.75,59.22,58.29,-1.55,-1.15
2022-05-02,3IINFOLTD.NS,738277.0,56.90,56.90,55.40,55.75,57.73,56.93,-1.49,-1.36
2022-05-04,3IINFOLTD.NS,738279.0,57.00,57.35,54.00,54.50,56.47,55.54,-1.26,-1.39


[*********************100%***********************]  1 of 1 completed
Checking 3MINDIA.NS.....


,Symbol,Date,Open,High,Low,Close,src1,src2,momm1,momm2
Date,,,,,,,,,,
2020-03-05,3MINDIA.NS,737489.0,22777.0,24349.900,22500.00,22885.449,NaN,NaN,NaN,NaN
2020-03-06,3MINDIA.NS,737490.0,22380.0,23638.850,21717.90,22601.850,NaN,NaN,NaN,NaN
2020-03-09,3MINDIA.NS,737493.0,22000.0,22749.000,20975.00,21038.449,NaN,NaN,NaN,NaN
2020-03-11,3MINDIA.NS,737495.0,21190.0,21285.150,20600.00,20683.350,NaN,NaN,NaN,NaN
2020-03-12,3MINDIA.NS,737496.0,20000.0,20000.000,18550.00,19057.801,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-29,3MINDIA.NS,738274.0,20190.0,20250.000,19831.00,19964.551,19898.87,19728.44,-146.04,-105.49
2022-05-02,3MINDIA.NS,738277.0,19806.6,20101.000,19600.00,19831.100,20032.29,19677.87,133.42,-50.57
2022-05-04,3MINDIA.NS,738279.0,19950.0,19950.000,18810.25,19150.301,19918.58,19526.59,-113.71,-151.28


[*********************100%***********************]  1 of 1 completed
Checking 5PAISA.NS.....


,Symbol,Date,Open,High,Low,Close,src1,src2,momm1,momm2
Date,,,,,,,,,,
2020-03-05,5PAISA.NS,737489.0,160.00,160.00,153.00,153.45,NaN,NaN,NaN,NaN
2020-03-06,5PAISA.NS,737490.0,151.00,157.50,146.80,152.25,NaN,NaN,NaN,NaN
2020-03-09,5PAISA.NS,737493.0,159.40,159.40,136.05,145.95,NaN,NaN,NaN,NaN
2020-03-11,5PAISA.NS,737495.0,151.75,155.00,147.00,149.45,NaN,NaN,NaN,NaN
2020-03-12,5PAISA.NS,737496.0,140.05,147.00,125.00,130.80,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-29,5PAISA.NS,738274.0,371.10,371.20,357.80,360.50,374.57,368.28,7.35,-3.97
2022-05-02,5PAISA.NS,738277.0,352.10,359.80,332.30,339.90,377.68,359.76,3.11,-8.52
2022-05-04,5PAISA.NS,738279.0,348.00,354.95,331.90,337.10,358.91,349.55,-18.77,-10.21
